# 掷硬币问题

投掷一颗硬币，观察硬币处于正面还是反面，假设硬币的投掷服从二项分布，使用最大似然参数估计法求解该问题的概率分布模型，其中二项分布中的$p$代表硬币朝上的概率。

给定数据，数据中仅包含$0,1$变量，$1$，代表硬币朝上，$0$代表硬币朝下，使用最大似然估计法给出推导过程，并使用`python`统计数据，给出参数估计结果。

---

根据要求，生成硬币数据，设定向上概率$p=0.52$。

In [2]:
import numpy as np
p = 0.52
n = 100000 # 模拟掷硬币100000次
X = np.random.choice(2, n, p=[1 - p, p])

查看数据并保存到`coin.npy`：

In [5]:
k = np.count_nonzero(X == 1)
print(f'Total: {n}, Up: {k}, Down: {n-k}')
np.save('coin.npy', X)

Total: 100000, Up: 52095, Down: 47905


我们可以看到，投掷100000次，有52095次是朝上的，符合先验的概率$p$。

下面我们进行最大似然的推导。$X$的先验分布为二项分布，即：
$$
\left\{ \begin{aligned}
	P\left( x=1 \right) &=p\\
	P\left( x=0 \right) &=1-p\\
\end{aligned} \right. 
$$
其中$p$是未知参数。实际过程中我们投掷$n$次，抛出$k$次朝上，由于每次抛硬币是相互独立的，不失一般性，我们假设前$k$次朝上。那么最大似然函数为
$$
\begin{align*}
\max  \prod\nolimits_{i=1}^n{P(x_i)} &= \max  \sum_{i=1}^n{\log P(x_i)} \\
&= \max  \sum_{i=1}^k{\begin{array}{c}
	\log P(x_i=1)\\
\end{array}}+\sum_{i=k+1}^n{\log}P\left( x_i=0 \right) 
\\
&= \max \left( k\log p+\left( n-k \right) \log \left( 1-p \right) \right) 
\end{align*}
$$
我们注意到最后的函数为凹函数，因此可以直接求解，令其对$p$导数为$0$可得$p=\frac{k}{n}=0.52095$。
我们也可以使用`cvxpy`进行求解：


In [6]:
import cvxpy as cp
p = cp.Variable()
obj = cp.Maximize(k * cp.log(p) + (n - k) * cp.log(1 - p))
constraints = [p >= 0, p <= 1]
prob = cp.Problem(obj, constraints)
prob.solve()

-69226.91185322001

输出求得的$p$值：

In [8]:
print(f'The value of p is {p.value: .5f}.')

The value of p is  0.52095.


故待求的抛硬币分布为
$$
\left\{ \begin{aligned}
	P\left( x=1 \right) &=0.52095\\
	P\left( x=0 \right) &=0.47905\\
\end{aligned} \right. 
$$
这与我们开始所设置的$p=0.52$相符合。